# Importing the Required Libraries

In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv3D,Conv2D, MaxPooling2D,TimeDistributed,LSTM,ConvLSTM2D
from keras.utils import np_utils

# Required Functions

Only chest sensor data

## Function for Loading Data

In [ ]:
# Loading data

def load_data(x):

  data_set = "/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/"
  sx_path = data_set + x
  with open(sx_path, 'rb') as file:
    sx_data = pickle.load(file, encoding='latin1')
  c_ax=sx_data['signal']['chest']['ACC'][0:,0]
  c_ay=sx_data['signal']['chest']['ACC'][0:,1]
  c_az=sx_data['signal']['chest']['ACC'][0:,2]
  c_ecg=sx_data['signal']['chest']['ECG'][:,0]
  c_emg=sx_data['signal']['chest']['EMG'][:,0]
  c_eda=sx_data['signal']['chest']['EDA'][:,0]
  c_temp=sx_data['signal']['chest']['Temp'][:,0]
  c_resp=sx_data['signal']['chest']['Resp'][:,0]
  w_label=sx_data['label']
  numpy_data=np.array([c_ax, c_ay, c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label])
  numpy_data=numpy_data.T
  chest_df = pd.DataFrame(data=numpy_data, 
                           columns=["c_ax", "c_ay", "c_az", "c_ecg","c_emg","c_eda","c_temp","c_resp","w_label"]) 
  chest_df['w_label'] = chest_df['w_label'].astype('int')
  df = chest_df.loc[chest_df['w_label'].isin([1,2,3])]
  return df



## Function for creating windows

In [ ]:

def window(data, size, shift):
  
  limit = 3500  # window size
  w = []   # windows
  w_count = 0 #no of windows
  j= 0
  for i in range(len(data)):
      res = data.iloc[j:size,:]
      if len(res.index) != limit:
         break
      else:
         res= res.T
         w.append(res)
         j = j + shift
         size = shift + size
         w_count = w_count + 1
  
  return w , w_count


## Function for making the data ready to be fed into the model

In [ ]:
# For making the data ready to be fed into the model

def data_ready(x_data , y_no, category):

  x = np.array(x_data)
  y = np.full( shape= y_no, fill_value= category, dtype=np.int)

  return x,y 

# S2

In [ ]:
s2_path = "S2/S2.pkl"
s2 = load_data(s2_path)
print(s2.head())
print()
print('Shape:')
print(s2.shape)
print()
print('dtypes:')
print(s2.dtypes)
print()
print('Value Counts:')
print(s2['w_label'].value_counts())
print()
s2_final = s2.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s2_final.head())

          c_ax    c_ay    c_az  ...     c_temp    c_resp  w_label
214583  0.8914 -0.1102 -0.2576  ...  29.083618  1.191711        1
214584  0.8926 -0.1086 -0.2544  ...  29.122437  1.139832        1
214585  0.8930 -0.1094 -0.2580  ...  29.115234  1.141357        1
214586  0.8934 -0.1082 -0.2538  ...  29.126709  1.155090        1
214587  0.8930 -0.1096 -0.2570  ...  29.100861  1.133728        1

[5 rows x 9 columns]

Shape:
(1484700, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    800800
2    430500
3    253400
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
214583  0.030945 -0.003708  5.710983  29.083618  1.191711        1
214584  0.033646 -0.014145  5.719376  29.122437  1.139832        1
214585  0.033005  0.010208  5.706406  29.115234  1.141357        1
214586  0.031815  0

In [ ]:
print('Baseline:')
s2_base = s2_final.loc[s2_final['w_label'].isin([1])]
print(s2_base.shape)
s2_basex = s2_base.drop('w_label' ,axis = 1)
print(s2_basex.shape)
print()
print('Stress:')
s2_stress = s2_final.loc[s2_final['w_label'].isin([2])]
print(s2_stress.shape)
s2_stressx = s2_stress.drop('w_label',axis = 1)
print(s2_stressx.shape)
print()
print('Amusement:')
s2_amuse = s2_final.loc[s2_final['w_label'].isin([3])]
print(s2_amuse.shape)
s2_amusex = s2_amuse.drop('w_label',axis =1)
print(s2_amusex.shape)

Baseline:
(800800, 6)
(800800, 5)

Stress:
(430500, 6)
(430500, 5)

Amusement:
(253400, 6)
(253400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s2_basexw, count = window(s2_basex , 3500 ,1400)
print(count)
print(len(s2_basexw))

s2_basexwa , s2_basey  = data_ready(s2_basexw , count , 1)
print(s2_basexwa.shape)
print(s2_basey.shape)

print()
print('Stress:')
s2_stressxw, count = window(s2_stressx , 3500 ,1400)
print(count)
print(len(s2_stressxw))
s2_stressxwa , s2_stressy  = data_ready(s2_stressxw , count , 2)
print(s2_stressxwa.shape)
print(s2_stressy.shape)

print()
print('Amusement:')
s2_amusexw, count = window(s2_amusex , 3500 ,1400)
print(count)
print(len(s2_amusexw))
s2_amusexwa , s2_amusey  = data_ready(s2_amusexw , count , 3)
print(s2_amusexwa.shape)
print(s2_amusey.shape)

Creating windows:

Baseline:
570
570
(570, 5, 3500)
(570,)

Stress:
306
306
(306, 5, 3500)
(306,)

Amusement:
179
179
(179, 5, 3500)
(179,)


In [ ]:
x_s2 = np.concatenate( (s2_basexwa, s2_stressxwa, s2_amusexwa) , axis = 0)
print(len(x_s2))
print(x_s2.shape)
s2_x =  x_s2.reshape(x_s2.shape[0], 5, 3500, 1)
print(s2_x.shape)

1055
(1055, 5, 3500)
(1055, 5, 3500, 1)


In [ ]:
y_s2 = np.concatenate( (s2_basey, s2_stressy, s2_amusey) , axis = 0)
s2_y = pd.DataFrame(y_s2)

print(s2_y.shape)
print()
print(s2_y.columns)
print(s2_y.value_counts())
print()
print(s2_y.head())

(1055, 1)

RangeIndex(start=0, stop=1, step=1)
1    570
2    306
3    179
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S3

In [ ]:
s3_path = "S3/S3.pkl"
s3 = load_data(s3_path)
print(s3.head())
print()
print('Shape:')
print(s3.shape)
print()
print('dtypes:')
print(s3.dtypes)
print()
print('Value Counts:')
print(s3['w_label'].value_counts())
print()
s3_final = s3.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s3_final.head())

          c_ax    c_ay    c_az  ...     c_temp     c_resp  w_label
246998  0.7866 -0.0598  0.5048  ...  32.989807  12.303162        1
246999  0.7910 -0.0616  0.5048  ...  33.030365  12.315369        1
247000  0.7930 -0.0626  0.5018  ...  33.042389  12.339783        1
247001  0.7930 -0.0620  0.5022  ...  33.034882  12.336731        1
247002  0.7920 -0.0604  0.5038  ...  33.045380  12.318420        1

[5 rows x 9 columns]

Shape:
(1508500, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    798000
2    448000
3    262500
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp     c_resp  w_label
246998 -0.191849  0.058731  7.222748  32.989807  12.303162        1
246999 -0.186630  0.056259  7.224274  33.030365  12.315369        1
247000 -0.179260 -0.063034  7.222748  33.042389  12.339783        1
247001 -0

In [ ]:
print('Baseline:')
s3_base = s3_final.loc[s3_final['w_label'].isin([1])]
print(s3_base.shape)
s3_basex = s3_base.drop('w_label' ,axis = 1)
print(s3_basex.shape)
print()
print('Stress:')
s3_stress = s3_final.loc[s3_final['w_label'].isin([2])]
print(s3_stress.shape)
s3_stressx = s3_stress.drop('w_label',axis = 1)
print(s3_stressx.shape)
print()
print('Amusement:')
s3_amuse = s3_final.loc[s3_final['w_label'].isin([3])]
print(s3_amuse.shape)
s3_amusex = s3_amuse.drop('w_label',axis =1)
print(s3_amusex.shape)

Baseline:
(798000, 6)
(798000, 5)

Stress:
(448000, 6)
(448000, 5)

Amusement:
(262500, 6)
(262500, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s3_basexw, count = window(s3_basex , 3500 ,1400)
print(count)
print(len(s3_basexw))

s3_basexwa , s3_basey  = data_ready(s3_basexw , count , 1)
print(s3_basexwa.shape)
print(s3_basey.shape)

print()
print('Stress:')
s3_stressxw, count = window(s3_stressx , 3500 ,1400)
print(count)
print(len(s3_stressxw))
s3_stressxwa , s3_stressy  = data_ready(s3_stressxw , count , 2)
print(s3_stressxwa.shape)
print(s3_stressy.shape)

print()
print('Amusement:')
s3_amusexw, count = window(s3_amusex , 3500 ,1400)
print(count)
print(len(s3_amusexw))
s3_amusexwa , s3_amusey  = data_ready(s3_amusexw , count , 3)
print(s3_amusexwa.shape)
print(s3_amusey.shape)

Creating windows:

Baseline:
568
568
(568, 5, 3500)
(568,)

Stress:
318
318
(318, 5, 3500)
(318,)

Amusement:
186
186
(186, 5, 3500)
(186,)


In [ ]:
x_s3 = np.concatenate( (s3_basexwa, s3_stressxwa, s3_amusexwa) , axis = 0)
print(len(x_s3))
print(x_s3.shape)
s3_x =  x_s3.reshape(x_s3.shape[0], 5, 3500, 1)
print(s3_x.shape)

1072
(1072, 5, 3500)
(1072, 5, 3500, 1)


In [ ]:
y_s3 = np.concatenate( (s3_basey, s3_stressy, s3_amusey) , axis = 0)
s3_y = pd.DataFrame(y_s3)

print(s3_y.shape)
print()
print(s3_y.columns)
print(s3_y.value_counts())
print()
print(s3_y.head())

(1072, 1)

RangeIndex(start=0, stop=1, step=1)
1    568
2    318
3    186
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S4

In [ ]:
s4_path = "S4/S4.pkl"
s4 = load_data(s4_path)
print(s4.head())
print()
print('Shape:')
print(s4.shape)
print()
print('dtypes:')
print(s4.dtypes)
print()
print('Value Counts:')
print(s4['w_label'].value_counts())
print()
s4_final = s4.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s4_final.head())

          c_ax    c_ay    c_az  ...     c_temp    c_resp  w_label
199705  0.6746 -0.2214 -0.6216  ...  32.681061 -2.406311        1
199706  0.6742 -0.2198 -0.6236  ...  32.694519 -2.383423        1
199707  0.6752 -0.2216 -0.6274  ...  32.657135 -2.360535        1
199708  0.6770 -0.2240 -0.6322  ...  32.691559 -2.345276        1
199709  0.6760 -0.2230 -0.6282  ...  32.678101 -2.320862        1

[5 rows x 9 columns]

Shape:
(1515501, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    810601
2    444500
3    260400
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
199705  0.009567  0.011215  1.343536  32.681061 -2.406311        1
199706  0.003525  0.021652  1.316452  32.694519 -2.383423        1
199707 -0.000046  0.030212  1.325989  32.657135 -2.360535        1
199708  0.001968  0

In [ ]:
print('Baseline:')
s4_base = s4_final.loc[s4_final['w_label'].isin([1])]
print(s4_base.shape)
s4_basex = s4_base.drop('w_label' ,axis = 1)
print(s4_basex.shape)
print()
print('Stress:')
s4_stress = s4_final.loc[s4_final['w_label'].isin([2])]
print(s4_stress.shape)
s4_stressx = s4_stress.drop('w_label',axis = 1)
print(s4_stressx.shape)
print()
print('Amusement:')
s4_amuse = s4_final.loc[s4_final['w_label'].isin([3])]
print(s4_amuse.shape)
s4_amusex = s4_amuse.drop('w_label',axis =1)
print(s4_amusex.shape)

Baseline:
(810601, 6)
(810601, 5)

Stress:
(444500, 6)
(444500, 5)

Amusement:
(260400, 6)
(260400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s4_basexw, count = window(s4_basex , 3500 ,1400)
print(count)
print(len(s4_basexw))

s4_basexwa , s4_basey  = data_ready(s4_basexw , count , 1)
print(s4_basexwa.shape)
print(s4_basey.shape)

print()
print('Stress:')
s4_stressxw, count = window(s4_stressx , 3500 ,1400)
print(count)
print(len(s4_stressxw))
s4_stressxwa , s4_stressy  = data_ready(s4_stressxw , count , 2)
print(s4_stressxwa.shape)
print(s4_stressy.shape)

print()
print('Amusement:')
s4_amusexw, count = window(s4_amusex , 3500 ,1400)
print(count)
print(len(s4_amusexw))
s4_amusexwa , s4_amusey  = data_ready(s4_amusexw , count , 3)
print(s4_amusexwa.shape)
print(s4_amusey.shape)

Creating windows:

Baseline:
577
577
(577, 5, 3500)
(577,)

Stress:
316
316
(316, 5, 3500)
(316,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s4 = np.concatenate( (s4_basexwa, s4_stressxwa, s4_amusexwa) , axis = 0)
print(len(x_s4))
print(x_s4.shape)
s4_x =  x_s4.reshape(x_s4.shape[0], 5, 3500, 1)
print(s4_x.shape)

1077
(1077, 5, 3500)
(1077, 5, 3500, 1)


In [ ]:
y_s4 = np.concatenate( (s4_basey, s4_stressy, s4_amusey) , axis = 0)
s4_y = pd.DataFrame(y_s4)

print(s4_y.shape)
print()
print(s4_y.columns)
print(s4_y.value_counts())
print()
print(s4_y.head())

(1077, 1)

RangeIndex(start=0, stop=1, step=1)
1    577
2    316
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S5

In [ ]:
s5_path = "S5/S5.pkl"
s5 = load_data(s5_path)
print(s5.head())
print()
print('Shape:')
print(s5.shape)
print()
print('dtypes:')
print(s5.dtypes)
print()
print('Value Counts:')
print(s5['w_label'].value_counts())
print()
s5_final = s5.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s5_final.head())

          c_ax    c_ay    c_az  ...     c_temp    c_resp  w_label
195560  0.8408 -0.0390 -0.3862  ...  34.251160  2.789307        1
195561  0.8418 -0.0422 -0.3862  ...  34.252716  2.891541        1
195562  0.8386 -0.0390 -0.3906  ...  34.249634  2.861023        1
195563  0.8406 -0.0394 -0.3914  ...  34.298553  2.833557        1
195564  0.8424 -0.0434 -0.3914  ...  34.326050  2.857971        1

[5 rows x 9 columns]

Shape:
(1551900, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    838600
2    451500
3    261800
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
195560 -0.028976 -0.007828  6.387711  34.251160  2.789307        1
195561 -0.029114 -0.003296  6.384277  34.252716  2.891541        1
195562 -0.030212 -0.005264  6.398392  34.249634  2.861023        1
195563 -0.030991 -0

In [ ]:
print('Baseline:')
s5_base = s5_final.loc[s5_final['w_label'].isin([1])]
print(s5_base.shape)
s5_basex = s5_base.drop('w_label' ,axis = 1)
print(s5_basex.shape)
print()
print('Stress:')
s5_stress = s5_final.loc[s5_final['w_label'].isin([2])]
print(s5_stress.shape)
s5_stressx = s5_stress.drop('w_label',axis = 1)
print(s5_stressx.shape)
print()
print('Amusement:')
s5_amuse = s5_final.loc[s5_final['w_label'].isin([3])]
print(s5_amuse.shape)
s5_amusex = s5_amuse.drop('w_label',axis =1)
print(s5_amusex.shape)

Baseline:
(838600, 6)
(838600, 5)

Stress:
(451500, 6)
(451500, 5)

Amusement:
(261800, 6)
(261800, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s5_basexw, count = window(s5_basex , 3500 ,1400)
print(count)
print(len(s5_basexw))

s5_basexwa , s5_basey  = data_ready(s5_basexw , count , 1)
print(s5_basexwa.shape)
print(s5_basey.shape)

print()
print('Stress:')
s5_stressxw, count = window(s5_stressx , 3500 ,1400)
print(count)
print(len(s5_stressxw))
s5_stressxwa , s5_stressy  = data_ready(s5_stressxw , count , 2)
print(s5_stressxwa.shape)
print(s5_stressy.shape)

print()
print('Amusement:')
s5_amusexw, count = window(s5_amusex , 3500 ,1400)
print(count)
print(len(s5_amusexw))
s5_amusexwa , s5_amusey  = data_ready(s5_amusexw , count , 3)
print(s5_amusexwa.shape)
print(s5_amusey.shape)

Creating windows:

Baseline:
597
597
(597, 5, 3500)
(597,)

Stress:
321
321
(321, 5, 3500)
(321,)

Amusement:
185
185
(185, 5, 3500)
(185,)


In [ ]:
x_s5 = np.concatenate( (s5_basexwa, s5_stressxwa, s5_amusexwa) , axis = 0)
print(len(x_s5))
print(x_s5.shape)
s5_x =  x_s5.reshape(x_s5.shape[0], 5, 3500, 1)
print(s5_x.shape)

1103
(1103, 5, 3500)
(1103, 5, 3500, 1)


In [ ]:
y_s5 = np.concatenate( (s5_basey, s5_stressy, s5_amusey) , axis = 0)
s5_y = pd.DataFrame(y_s5)

print(s5_y.shape)
print()
print(s5_y.columns)
print(s5_y.value_counts())
print()
print(s5_y.head())

(1103, 1)

RangeIndex(start=0, stop=1, step=1)
1    597
2    321
3    185
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S6

In [ ]:
s6_path = "S6/S6.pkl"
s6 = load_data(s6_path)
print(s6.head())
print()
print('Shape:')
print(s6.shape)
print()
print('dtypes:')
print(s6.dtypes)
print()
print('Value Counts:')
print(s6['w_label'].value_counts())
print()
s6_final = s6.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s6_final.head())

          c_ax    c_ay    c_az  ...     c_temp    c_resp  w_label
449887  0.6280 -0.0082 -0.7272  ...  33.432587  4.010010        1
449888  0.6276 -0.0056 -0.7262  ...  33.459747  4.039001        1
449889  0.6270 -0.0054 -0.7200  ...  33.452240  3.993225        1
449890  0.6266 -0.0070 -0.7174  ...  33.453735  4.002380        1
449891  0.6242 -0.0078 -0.7202  ...  33.426544  3.996277        1

[5 rows x 9 columns]

Shape:
(1541400, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826000
2    455000
3    260400
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
449887  0.016937  0.134720  9.645462  33.432587  4.010010        1
449888  0.017853  0.216248  9.645462  33.459747  4.039001        1
449889  0.016525  0.142136  9.646606  33.452240  3.993225        1
449890  0.012863  0

In [ ]:
print('Baseline:')
s6_base = s6_final.loc[s6_final['w_label'].isin([1])]
print(s6_base.shape)
s6_basex = s6_base.drop('w_label' ,axis = 1)
print(s6_basex.shape)
print()
print('Stress:')
s6_stress = s6_final.loc[s6_final['w_label'].isin([2])]
print(s6_stress.shape)
s6_stressx = s6_stress.drop('w_label',axis = 1)
print(s6_stressx.shape)
print()
print('Amusement:')
s6_amuse = s6_final.loc[s6_final['w_label'].isin([3])]
print(s6_amuse.shape)
s6_amusex = s6_amuse.drop('w_label',axis =1)
print(s6_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(455000, 6)
(455000, 5)

Amusement:
(260400, 6)
(260400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s6_basexw, count = window(s6_basex , 3500 ,1400)
print(count)
print(len(s6_basexw))

s6_basexwa , s6_basey  = data_ready(s6_basexw , count , 1)
print(s6_basexwa.shape)
print(s6_basey.shape)

print()
print('Stress:')
s6_stressxw, count = window(s6_stressx , 3500 ,1400)
print(count)
print(len(s6_stressxw))
s6_stressxwa , s6_stressy  = data_ready(s6_stressxw , count , 2)
print(s6_stressxwa.shape)
print(s6_stressy.shape)

print()
print('Amusement:')
s6_amusexw, count = window(s6_amusex , 3500 ,1400)
print(count)
print(len(s6_amusexw))
s6_amusexwa , s6_amusey  = data_ready(s6_amusexw , count , 3)
print(s6_amusexwa.shape)
print(s6_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
323
323
(323, 5, 3500)
(323,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s6 = np.concatenate( (s6_basexwa, s6_stressxwa, s6_amusexwa) , axis = 0)
print(len(x_s6))
print(x_s6.shape)
s6_x =  x_s6.reshape(x_s6.shape[0], 5, 3500, 1)
print(s6_x.shape)

1095
(1095, 5, 3500)
(1095, 5, 3500, 1)


In [ ]:
y_s6 = np.concatenate( (s6_basey, s6_stressy, s6_amusey) , axis = 0)
s6_y = pd.DataFrame(y_s6)

print(s6_y.shape)
print()
print(s6_y.columns)
print(s6_y.value_counts())
print()
print(s6_y.head())

(1095, 1)

RangeIndex(start=0, stop=1, step=1)
1    588
2    323
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S7

In [ ]:
s7_path = "S7/S7.pkl"
s7 = load_data(s7_path)
print(s7.head())
print()
print('Shape:')
print(s7.shape)
print()
print('dtypes:')
print(s7.dtypes)
print()
print('Value Counts:')
print(s7['w_label'].value_counts())
print()
s7_final = s7.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s7_final.head())

         c_ax    c_ay    c_az  ...     c_temp    c_resp  w_label
70442  0.9002 -0.0758 -0.2070  ...  34.202301 -0.292969        1
70443  0.9018 -0.0744 -0.2090  ...  34.216064 -0.300598        1
70444  0.8996 -0.0734 -0.2078  ...  34.217560 -0.297546        1
70445  0.9014 -0.0750 -0.2088  ...  34.222168 -0.285339        1
70446  0.8964 -0.0726 -0.2074  ...  34.197754 -0.276184        1

[5 rows x 9 columns]

Shape:
(1538601, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    830200
2    448000
3    260401
Name: w_label, dtype: int64

          c_ecg     c_emg      c_eda     c_temp    c_resp  w_label
70442 -0.013504 -0.031540  12.968826  34.202301 -0.292969        1
70443 -0.014694 -0.018723  12.967300  34.216064 -0.300598        1
70444 -0.014603 -0.012955  12.965775  34.217560 -0.297546        1
70445 -0.012955  0.005630

In [ ]:
print('Baseline:')
s7_base = s7_final.loc[s7_final['w_label'].isin([1])]
print(s7_base.shape)
s7_basex = s7_base.drop('w_label' ,axis = 1)
print(s7_basex.shape)
print()
print('Stress:')
s7_stress = s7_final.loc[s7_final['w_label'].isin([2])]
print(s7_stress.shape)
s7_stressx = s7_stress.drop('w_label',axis = 1)
print(s7_stressx.shape)
print()
print('Amusement:')
s7_amuse = s7_final.loc[s7_final['w_label'].isin([3])]
print(s7_amuse.shape)
s7_amusex = s7_amuse.drop('w_label',axis =1)
print(s7_amusex.shape)

Baseline:
(830200, 6)
(830200, 5)

Stress:
(448000, 6)
(448000, 5)

Amusement:
(260401, 6)
(260401, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s7_basexw, count = window(s7_basex , 3500 ,1400)
print(count)
print(len(s7_basexw))

s7_basexwa , s7_basey  = data_ready(s7_basexw , count , 1)
print(s7_basexwa.shape)
print(s7_basey.shape)

print()
print('Stress:')
s7_stressxw, count = window(s7_stressx , 3500 ,1400)
print(count)
print(len(s7_stressxw))
s7_stressxwa , s7_stressy  = data_ready(s7_stressxw , count , 2)
print(s7_stressxwa.shape)
print(s7_stressy.shape)

print()
print('Amusement:')
s7_amusexw, count = window(s7_amusex , 3500 ,1400)
print(count)
print(len(s7_amusexw))
s7_amusexwa , s7_amusey  = data_ready(s7_amusexw , count , 3)
print(s7_amusexwa.shape)
print(s7_amusey.shape)

Creating windows:

Baseline:
591
591
(591, 5, 3500)
(591,)

Stress:
318
318
(318, 5, 3500)
(318,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s7 = np.concatenate( (s7_basexwa, s7_stressxwa, s7_amusexwa) , axis = 0)
print(len(x_s7))
print(x_s7.shape)
s7_x =  x_s7.reshape(x_s7.shape[0], 5, 3500, 1)
print(s7_x.shape)

1093
(1093, 5, 3500)
(1093, 5, 3500, 1)


In [ ]:
y_s7 = np.concatenate( (s7_basey, s7_stressy, s7_amusey) , axis = 0)
s7_y = pd.DataFrame(y_s7)

print(s7_y.shape)
print()
print(s7_y.columns)
print(s7_y.value_counts())
print()
print(s7_y.head())

(1093, 1)

RangeIndex(start=0, stop=1, step=1)
1    591
2    318
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S8

In [ ]:
s8_path = "S8/S8.pkl"
s8 = load_data(s8_path)
print(s8.head())
print()
print('Shape:')
print(s8.shape)
print()
print('dtypes:')
print(s8.dtypes)
print()
print('Value Counts:')
print(s8['w_label'].value_counts())
print()
s8_final = s8.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s8_final.head())

          c_ax    c_ay    c_az  ...     c_temp    c_resp  w_label
110972  0.7112 -0.1910 -0.6150  ...  34.315338  0.465393        1
110973  0.7112 -0.1934 -0.6150  ...  34.240479  0.152588        1
110974  0.7114 -0.1932 -0.6130  ...  34.301605  0.471497        1
110975  0.7108 -0.1922 -0.6122  ...  34.234375  0.482178        1
110976  0.7082 -0.1922 -0.6078  ...  34.300049  0.506592        1

[5 rows x 9 columns]

Shape:
(1546299, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    818300
2    469000
3    258999
Name: w_label, dtype: int64

           c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
110972 -0.015884  0.005951  4.188919  34.315338  0.465393        1
110973 -0.017349 -0.013412  4.185104  34.240479  0.152588        1
110974 -0.014511 -0.014786  4.186630  34.301605  0.471497        1
110975 -0.010391 -0

In [ ]:
print('Baseline:')
s8_base = s8_final.loc[s8_final['w_label'].isin([1])]
print(s8_base.shape)
s8_basex = s8_base.drop('w_label' ,axis = 1)
print(s8_basex.shape)
print()
print('Stress:')
s8_stress = s8_final.loc[s8_final['w_label'].isin([2])]
print(s8_stress.shape)
s8_stressx = s8_stress.drop('w_label',axis = 1)
print(s8_stressx.shape)
print()
print('Amusement:')
s8_amuse = s8_final.loc[s8_final['w_label'].isin([3])]
print(s8_amuse.shape)
s8_amusex = s8_amuse.drop('w_label',axis =1)
print(s8_amusex.shape)

Baseline:
(818300, 6)
(818300, 5)

Stress:
(469000, 6)
(469000, 5)

Amusement:
(258999, 6)
(258999, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s8_basexw, count = window(s8_basex , 3500 ,1400)
print(count)
print(len(s8_basexw))

s8_basexwa , s8_basey  = data_ready(s8_basexw , count , 1)
print(s8_basexwa.shape)
print(s8_basey.shape)

print()
print('Stress:')
s8_stressxw, count = window(s8_stressx , 3500 ,1400)
print(count)
print(len(s8_stressxw))
s8_stressxwa , s8_stressy  = data_ready(s8_stressxw , count , 2)
print(s8_stressxwa.shape)
print(s8_stressy.shape)

print()
print('Amusement:')
s8_amusexw, count = window(s8_amusex , 3500 ,1400)
print(count)
print(len(s8_amusexw))
s8_amusexwa , s8_amusey  = data_ready(s8_amusexw , count , 3)
print(s8_amusexwa.shape)
print(s8_amusey.shape)

Creating windows:

Baseline:
583
583
(583, 5, 3500)
(583,)

Stress:
333
333
(333, 5, 3500)
(333,)

Amusement:
183
183
(183, 5, 3500)
(183,)


In [ ]:
x_s8 = np.concatenate( (s8_basexwa, s8_stressxwa, s8_amusexwa) , axis = 0)
print(len(x_s8))
print(x_s8.shape)
s8_x =  x_s8.reshape(x_s8.shape[0], 5, 3500, 1)
print(s8_x.shape)

1099
(1099, 5, 3500)
(1099, 5, 3500, 1)


In [ ]:
y_s8 = np.concatenate( (s8_basey, s8_stressy, s8_amusey) , axis = 0)
s8_y = pd.DataFrame(y_s8)

print(s8_y.shape)
print()
print(s8_y.columns)
print(s8_y.value_counts())
print()
print(s8_y.head())

(1099, 1)

RangeIndex(start=0, stop=1, step=1)
1    583
2    333
3    183
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


# S9

In [ ]:
s9_path = "S9/S9.pkl"
s9 = load_data(s9_path)
print(s9.head())
print()
print('Shape:')
print(s9.shape)
print()
print('dtypes:')
print(s9.dtypes)
print()
print('Value Counts:')
print(s9['w_label'].value_counts())
print()
s9_final = s9.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s9_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
52672  0.9128 -0.0858 -0.0858  0.172073  ...  2.141571  33.780762  6.053162        1
52673  0.9122 -0.0866 -0.0862  0.157242  ...  2.154922  33.774689  6.065369        1
52674  0.9140 -0.0890 -0.0838  0.143967  ...  2.141571  33.723114  6.071472        1
52675  0.9146 -0.0890 -0.0822  0.144882  ...  2.159119  33.789917  6.065369        1
52676  0.9150 -0.0878 -0.0798  0.149963  ...  2.142334  33.782318  6.074524        1

[5 rows x 9 columns]

Shape:
(1537900, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826000
2    451500
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
52672  0.172073 -0.013321  2.141571  33.780762  6.053162        1
52673  0.157242  0.006180  2.154922  33.7

In [ ]:
print('Baseline:')
s9_base = s9_final.loc[s9_final['w_label'].isin([1])]
print(s9_base.shape)
s9_basex = s9_base.drop('w_label' ,axis = 1)
print(s9_basex.shape)
print()
print('Stress:')
s9_stress = s9_final.loc[s9_final['w_label'].isin([2])]
print(s9_stress.shape)
s9_stressx = s9_stress.drop('w_label',axis = 1)
print(s9_stressx.shape)
print()
print('Amusement:')
s9_amuse = s9_final.loc[s9_final['w_label'].isin([3])]
print(s9_amuse.shape)
s9_amusex = s9_amuse.drop('w_label',axis =1)
print(s9_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(451500, 6)
(451500, 5)

Amusement:
(260400, 6)
(260400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s9_basexw, count = window(s9_basex , 3500 ,1400)
print(count)
print(len(s9_basexw))

s9_basexwa , s9_basey  = data_ready(s9_basexw , count , 1)
print(s9_basexwa.shape)
print(s9_basey.shape)

print()
print('Stress:')
s9_stressxw, count = window(s9_stressx , 3500 ,1400)
print(count)
print(len(s9_stressxw))
s9_stressxwa , s9_stressy  = data_ready(s9_stressxw , count , 2)
print(s9_stressxwa.shape)
print(s9_stressy.shape)

print()
print('Amusement:')
s9_amusexw, count = window(s9_amusex , 3500 ,1400)
print(count)
print(len(s9_amusexw))
s9_amusexwa , s9_amusey  = data_ready(s9_amusexw , count , 3)
print(s9_amusexwa.shape)
print(s9_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
321
321
(321, 5, 3500)
(321,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s9 = np.concatenate( (s9_basexwa, s9_stressxwa, s9_amusexwa) , axis = 0)
print(len(x_s9))
print(x_s9.shape)
s9_x =  x_s9.reshape(x_s9.shape[0], 5, 3500, 1)
print(s9_x.shape)

1093
(1093, 5, 3500)
(1093, 5, 3500, 1)


In [ ]:
y_s9 = np.concatenate( (s9_basey, s9_stressy, s9_amusey) , axis = 0)
s9_y = pd.DataFrame(y_s9)

print(s9_y.shape)
print()
print(s9_y.columns)
print(s9_y.value_counts())
print()
print(s9_y.head())

(1093, 1)

RangeIndex(start=0, stop=1, step=1)
1    588
2    321
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S10

In [ ]:
s10_path = "S10/S10.pkl"
s10 = load_data(s10_path)
print(s10.head())
print()
print('Shape:')
print(s10.shape)
print()
print('dtypes:')
print(s10.dtypes)
print()
print('Value Counts:')
print(s10['w_label'].value_counts())
print()
s10_final = s10.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s10_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
65000  0.8986  0.0874 -0.2018 -0.006775  ...  0.730515  33.891632  0.685120        1
65001  0.8994  0.0882 -0.2008 -0.002884  ...  0.745773  33.881012  0.712585        1
65002  0.8978  0.0870 -0.2022 -0.002289  ...  0.755310  33.873383  0.737000        1
65003  0.9000  0.0866 -0.1994  0.002792  ...  0.722885  33.893158  0.740051        1
65004  0.9002  0.0870 -0.1990  0.008560  ...  0.728989  33.900757  0.733948        1

[5 rows x 9 columns]

Shape:
(1593900, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826000
2    507500
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
65000 -0.006775  0.013596  0.730515  33.891632  0.685120        1
65001 -0.002884  0.015198  0.745773  33.8

In [ ]:
print('Baseline:')
s10_base = s10_final.loc[s10_final['w_label'].isin([1])]
print(s10_base.shape)
s10_basex = s10_base.drop('w_label' ,axis = 1)
print(s10_basex.shape)
print()
print('Stress:')
s10_stress = s10_final.loc[s10_final['w_label'].isin([2])]
print(s10_stress.shape)
s10_stressx = s10_stress.drop('w_label',axis = 1)
print(s10_stressx.shape)
print()
print('Amusement:')
s10_amuse = s10_final.loc[s10_final['w_label'].isin([3])]
print(s10_amuse.shape)
s10_amusex = s10_amuse.drop('w_label',axis =1)
print(s10_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(507500, 6)
(507500, 5)

Amusement:
(260400, 6)
(260400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s10_basexw, count = window(s10_basex , 3500 ,1400)
print(count)
print(len(s10_basexw))

s10_basexwa , s10_basey  = data_ready(s10_basexw , count , 1)
print(s10_basexwa.shape)
print(s10_basey.shape)

print()
print('Stress:')
s10_stressxw, count = window(s10_stressx , 3500 ,1400)
print(count)
print(len(s10_stressxw))
s10_stressxwa , s10_stressy  = data_ready(s10_stressxw , count , 2)
print(s10_stressxwa.shape)
print(s10_stressy.shape)

print()
print('Amusement:')
s10_amusexw, count = window(s10_amusex , 3500 ,1400)
print(count)
print(len(s10_amusexw))
s10_amusexwa , s10_amusey  = data_ready(s10_amusexw , count , 3)
print(s10_amusexwa.shape)
print(s10_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
361
361
(361, 5, 3500)
(361,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s10 = np.concatenate( (s10_basexwa, s10_stressxwa, s10_amusexwa) , axis = 0)
print(len(x_s10))
print(x_s10.shape)
s10_x =  x_s10.reshape(x_s10.shape[0], 5, 3500, 1)
print(s10_x.shape)

1133
(1133, 5, 3500)
(1133, 5, 3500, 1)


In [ ]:
y_s10 = np.concatenate( (s10_basey, s10_stressy, s10_amusey) , axis = 0)
s10_y = pd.DataFrame(y_s10)

print(s10_y.shape)
print()
print(s10_y.columns)
print(s10_y.value_counts())
print()
print(s10_y.head())

(1133, 1)

RangeIndex(start=0, stop=1, step=1)
1    588
2    361
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


# S11

In [ ]:
s11_path = "S11/S11.pkl"
s11 = load_data(s11_path)
print(s11.head())
print()
print('Shape:')
print(s11.shape)
print()
print('dtypes:')
print(s11.dtypes)
print()
print('Value Counts:')
print(s11['w_label'].value_counts())
print()
s11_final = s11.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s11_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
59937  0.8968  0.0414 -0.1698 -0.017395  ...  6.644821  33.677673  1.760864        1
59938  0.8978  0.0418 -0.1714 -0.012772  ...  6.648254  33.612549  1.785278        1
59939  0.9044  0.0406 -0.1708 -0.007599  ...  6.637192  33.665527  1.788330        1
59940  0.9000  0.0414 -0.1766 -0.010574  ...  6.637573  33.657959  1.777649        1
59941  0.8978  0.0408 -0.1698 -0.015152  ...  6.638718  33.665527  1.817322        1

[5 rows x 9 columns]

Shape:
(1559600, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826000
2    476000
3    257600
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
59937 -0.017395  0.002014  6.644821  33.677673  1.760864        1
59938 -0.012772  0.010117  6.648254  33.6

In [ ]:
print('Baseline:')
s11_base = s11_final.loc[s11_final['w_label'].isin([1])]
print(s11_base.shape)
s11_basex = s11_base.drop('w_label' ,axis = 1)
print(s11_basex.shape)
print()
print('Stress:')
s11_stress = s11_final.loc[s11_final['w_label'].isin([2])]
print(s11_stress.shape)
s11_stressx = s11_stress.drop('w_label',axis = 1)
print(s11_stressx.shape)
print()
print('Amusement:')
s11_amuse = s11_final.loc[s11_final['w_label'].isin([3])]
print(s11_amuse.shape)
s11_amusex = s11_amuse.drop('w_label',axis =1)
print(s11_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(476000, 6)
(476000, 5)

Amusement:
(257600, 6)
(257600, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s11_basexw, count = window(s11_basex , 3500 ,1400)
print(count)
print(len(s11_basexw))

s11_basexwa , s11_basey  = data_ready(s11_basexw , count , 1)
print(s11_basexwa.shape)
print(s11_basey.shape)

print()
print('Stress:')
s11_stressxw, count = window(s11_stressx , 3500 ,1400)
print(count)
print(len(s11_stressxw))
s11_stressxwa , s11_stressy  = data_ready(s11_stressxw , count , 2)
print(s11_stressxwa.shape)
print(s11_stressy.shape)

print()
print('Amusement:')
s11_amusexw, count = window(s11_amusex , 3500 ,1400)
print(count)
print(len(s11_amusexw))
s11_amusexwa , s11_amusey  = data_ready(s11_amusexw , count , 3)
print(s11_amusexwa.shape)
print(s11_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
338
338
(338, 5, 3500)
(338,)

Amusement:
182
182
(182, 5, 3500)
(182,)


In [ ]:
x_s11 = np.concatenate( (s11_basexwa, s11_stressxwa, s11_amusexwa) , axis = 0)
print(len(x_s11))
print(x_s11.shape)
s11_x =  x_s11.reshape(x_s11.shape[0], 5, 3500, 1)
print(s11_x.shape)

1108
(1108, 5, 3500)
(1108, 5, 3500, 1)


In [ ]:
y_s11 = np.concatenate( (s11_basey, s11_stressy, s11_amusey) , axis = 0)
s11_y = pd.DataFrame(y_s11)

print(s11_y.shape)
print()
print(s11_y.columns)
print(s11_y.value_counts())
print()
print(s11_y.head())

(1108, 1)

RangeIndex(start=0, stop=1, step=1)
1    588
2    338
3    182
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S13

In [ ]:
s13_path = "S13/S13.pkl"
s13 = load_data(s13_path)
print(s13.head())
print()
print('Shape:')
print(s13.shape)
print()
print('dtypes:')
print(s13.dtypes)
print()
print('Value Counts:')
print(s13['w_label'].value_counts())
print()
s13_final = s13.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s13_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
61534  0.8978 -0.0228  0.0586 -0.206268  ...  2.383423  32.814331  0.721741        1
61535  0.9018 -0.0222  0.0564 -0.237213  ...  2.416992  32.830811  0.750732        1
61536  0.9042 -0.0226  0.0568 -0.269028  ...  2.375793  32.794861  0.776672        1
61537  0.9078 -0.0246  0.0606 -0.287247  ...  2.373123  32.832306  0.762939        1
61538  0.9106 -0.0298  0.0570 -0.296310  ...  2.376175  32.806824  0.787354        1

[5 rows x 9 columns]

Shape:
(1558201, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826001
2    464800
3    267400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
61534 -0.206268 -0.003983  2.383423  32.814331  0.721741        1
61535 -0.237213  0.040970  2.416992  32.8

In [ ]:
print('Baseline:')
s13_base = s13_final.loc[s13_final['w_label'].isin([1])]
print(s13_base.shape)
s13_basex = s13_base.drop('w_label' ,axis = 1)
print(s13_basex.shape)
print()
print('Stress:')
s13_stress = s13_final.loc[s13_final['w_label'].isin([2])]
print(s13_stress.shape)
s13_stressx = s13_stress.drop('w_label',axis = 1)
print(s13_stressx.shape)
print()
print('Amusement:')
s13_amuse = s13_final.loc[s13_final['w_label'].isin([3])]
print(s13_amuse.shape)
s13_amusex = s13_amuse.drop('w_label',axis =1)
print(s13_amusex.shape)

Baseline:
(826001, 6)
(826001, 5)

Stress:
(464800, 6)
(464800, 5)

Amusement:
(267400, 6)
(267400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s13_basexw, count = window(s13_basex , 3500 ,1400)
print(count)
print(len(s13_basexw))

s13_basexwa , s13_basey  = data_ready(s13_basexw , count , 1)
print(s13_basexwa.shape)
print(s13_basey.shape)

print()
print('Stress:')
s13_stressxw, count = window(s13_stressx , 3500 ,1400)
print(count)
print(len(s13_stressxw))
s13_stressxwa , s13_stressy  = data_ready(s13_stressxw , count , 2)
print(s13_stressxwa.shape)
print(s13_stressy.shape)

print()
print('Amusement:')
s13_amusexw, count = window(s13_amusex , 3500 ,1400)
print(count)
print(len(s13_amusexw))
s13_amusexwa , s13_amusey  = data_ready(s13_amusexw , count , 3)
print(s13_amusexwa.shape)
print(s13_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
330
330
(330, 5, 3500)
(330,)

Amusement:
189
189
(189, 5, 3500)
(189,)


In [ ]:
x_s13 = np.concatenate( (s13_basexwa, s13_stressxwa, s13_amusexwa) , axis = 0)
print(len(x_s13))
print(x_s13.shape)
s13_x =  x_s13.reshape(x_s13.shape[0], 5, 3500, 1)
print(s13_x.shape)

1107
(1107, 5, 3500)
(1107, 5, 3500, 1)


In [ ]:
y_s13 = np.concatenate( (s13_basey, s13_stressy, s13_amusey) , axis = 0)
s13_y = pd.DataFrame(y_s13)

print(s13_y.shape)
print()
print(s13_y.columns)
print(s13_y.value_counts())
print()
print(s13_y.head())

(1107, 1)

RangeIndex(start=0, stop=1, step=1)
1    588
2    330
3    189
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


# S14

In [ ]:
s14_path = "S14/S14.pkl"
s14 = load_data(s14_path)
print(s14.head())
print()
print('Shape:')
print(s14.shape)
print()
print('dtypes:')
print(s14.dtypes)
print()
print('Value Counts:')
print(s14['w_label'].value_counts())
print()
s14_final = s14.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s14_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
18337  0.7342 -0.0674 -0.5618  1.499222  ...  2.809906  32.851776  2.146912        1
18338  0.7350 -0.0688 -0.5622  1.499222  ...  2.825165  32.929749  1.878357        1
18339  0.7330 -0.0662 -0.5638  1.499084  ...  2.797699  32.845764  2.174377        1
18340  0.7348 -0.0686 -0.5646  1.498947  ...  2.817535  32.862244  2.165222        1
18341  0.7354 -0.0696 -0.5642  1.499130  ...  2.827072  32.866760  2.195740        1

[5 rows x 9 columns]

Shape:
(1558901, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826000
2    472500
3    260401
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
18337  1.499222 -0.011673  2.809906  32.851776  2.146912        1
18338  1.499222 -0.015060  2.825165  32.9

In [ ]:
print('Baseline:')
s14_base = s14_final.loc[s14_final['w_label'].isin([1])]
print(s14_base.shape)
s14_basex = s14_base.drop('w_label' ,axis = 1)
print(s14_basex.shape)
print()
print('Stress:')
s14_stress = s14_final.loc[s14_final['w_label'].isin([2])]
print(s14_stress.shape)
s14_stressx = s14_stress.drop('w_label',axis = 1)
print(s14_stressx.shape)
print()
print('Amusement:')
s14_amuse = s14_final.loc[s14_final['w_label'].isin([3])]
print(s14_amuse.shape)
s14_amusex = s14_amuse.drop('w_label',axis =1)
print(s14_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(472500, 6)
(472500, 5)

Amusement:
(260401, 6)
(260401, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s14_basexw, count = window(s14_basex , 3500 ,1400)
print(count)
print(len(s14_basexw))

s14_basexwa , s14_basey  = data_ready(s14_basexw , count , 1)
print(s14_basexwa.shape)
print(s14_basey.shape)

print()
print('Stress:')
s14_stressxw, count = window(s14_stressx , 3500 ,1400)
print(count)
print(len(s14_stressxw))
s14_stressxwa , s14_stressy  = data_ready(s14_stressxw , count , 2)
print(s14_stressxwa.shape)
print(s14_stressy.shape)

print()
print('Amusement:')
s14_amusexw, count = window(s14_amusex , 3500 ,1400)
print(count)
print(len(s14_amusexw))
s14_amusexwa , s14_amusey  = data_ready(s14_amusexw , count , 3)
print(s14_amusexwa.shape)
print(s14_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
336
336
(336, 5, 3500)
(336,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s14 = np.concatenate( (s14_basexwa, s14_stressxwa, s14_amusexwa) , axis = 0)
print(len(x_s14))
print(x_s14.shape)
s14_x =  x_s14.reshape(x_s14.shape[0], 5, 3500, 1)
print(s14_x.shape)

1108
(1108, 5, 3500)
(1108, 5, 3500, 1)


In [ ]:
y_s14 = np.concatenate( (s14_basey, s14_stressy, s14_amusey) , axis = 0)
s14_y = pd.DataFrame(y_s14)

print(s14_y.shape)
print()
print(s14_y.columns)
print(s14_y.value_counts())
print()
print(s14_y.head())

(1108, 1)

RangeIndex(start=0, stop=1, step=1)
1    588
2    336
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


# S15

In [ ]:
s15_path = "S15/S15.pkl"
s15 = load_data(s15_path)
print(s15.head())
print()
print('Shape:')
print(s15.shape)
print()
print('dtypes:')
print(s15.dtypes)
print()
print('Value Counts:')
print(s15['w_label'].value_counts())
print()
s15_final = s15.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s15_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
77161  0.9086  0.0728 -0.1022 -0.178162  ...  1.413727  33.535339 -2.075195        1
77162  0.9110  0.0762 -0.1038 -0.180222  ...  1.419830  33.535339 -2.064514        1
77163  0.9102  0.0738 -0.1030 -0.183334  ...  1.416016  33.532318 -2.098083        1
77164  0.9090  0.0748 -0.1010 -0.187088  ...  1.424026  33.536865 -2.069092        1
77165  0.9106  0.0744 -0.1014 -0.186768  ...  1.426697  33.541412 -2.067566        1

[5 rows x 9 columns]

Shape:
(1563100, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    822500
2    480200
3    260400
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
77161 -0.178162 -0.026321  1.413727  33.535339 -2.075195        1
77162 -0.180222 -0.026459  1.419830  33.5

In [ ]:
print('Baseline:')
s15_base = s15_final.loc[s15_final['w_label'].isin([1])]
print(s15_base.shape)
s15_basex = s15_base.drop('w_label' ,axis = 1)
print(s15_basex.shape)
print()
print('Stress:')
s15_stress = s15_final.loc[s15_final['w_label'].isin([2])]
print(s15_stress.shape)
s15_stressx = s15_stress.drop('w_label',axis = 1)
print(s15_stressx.shape)
print()
print('Amusement:')
s15_amuse = s15_final.loc[s15_final['w_label'].isin([3])]
print(s15_amuse.shape)
s15_amusex = s15_amuse.drop('w_label',axis =1)
print(s15_amusex.shape)

Baseline:
(822500, 6)
(822500, 5)

Stress:
(480200, 6)
(480200, 5)

Amusement:
(260400, 6)
(260400, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s15_basexw, count = window(s15_basex , 3500 ,1400)
print(count)
print(len(s15_basexw))

s15_basexwa , s15_basey  = data_ready(s15_basexw , count , 1)
print(s15_basexwa.shape)
print(s15_basey.shape)

print()
print('Stress:')
s15_stressxw, count = window(s15_stressx , 3500 ,1400)
print(count)
print(len(s15_stressxw))
s15_stressxwa , s15_stressy  = data_ready(s15_stressxw , count , 2)
print(s15_stressxwa.shape)
print(s15_stressy.shape)

print()
print('Amusement:')
s15_amusexw, count = window(s15_amusex , 3500 ,1400)
print(count)
print(len(s15_amusexw))
s15_amusexwa , s15_amusey  = data_ready(s15_amusexw , count , 3)
print(s15_amusexwa.shape)
print(s15_amusey.shape)

Creating windows:

Baseline:
586
586
(586, 5, 3500)
(586,)

Stress:
341
341
(341, 5, 3500)
(341,)

Amusement:
184
184
(184, 5, 3500)
(184,)


In [ ]:
x_s15 = np.concatenate( (s15_basexwa, s15_stressxwa, s15_amusexwa) , axis = 0)
print(len(x_s15))
print(x_s15.shape)
s15_x =  x_s15.reshape(x_s15.shape[0], 5, 3500, 1)
print(s15_x.shape)

1111
(1111, 5, 3500)
(1111, 5, 3500, 1)


In [ ]:
y_s15 = np.concatenate( (s15_basey, s15_stressy, s15_amusey) , axis = 0)
s15_y = pd.DataFrame(y_s15)

print(s15_y.shape)
print()
print(s15_y.columns)
print(s15_y.value_counts())
print()
print(s15_y.head())

(1111, 1)

RangeIndex(start=0, stop=1, step=1)
1    586
2    341
3    184
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


#S16

In [ ]:
s16_path = "S16/S16.pkl"
s16 = load_data(s16_path)
print(s16.head())
print()
print('Shape:')
print(s16.shape)
print()
print('dtypes:')
print(s16.dtypes)
print()
print('Value Counts:')
print(s16['w_label'].value_counts())
print()
s16_final = s16.drop(['c_ax','c_ay','c_az'] , axis = 1)
print(s16_final.head())

         c_ax    c_ay    c_az     c_ecg  ...     c_eda     c_temp    c_resp  w_label
68859  0.9022  0.0442 -0.0654  0.030533  ...  0.988770  33.376709  6.391907        1
68860  0.9048  0.0438 -0.0648  0.044449  ...  0.972366  33.363129  6.391907        1
68861  0.9070  0.0418 -0.0626  0.050903  ...  0.966263  33.376709  6.404114        1
68862  0.9072  0.0390 -0.0594  0.051590  ...  1.010132  33.379761  6.398010        1
68863  0.9054  0.0408 -0.0578  0.052917  ...  0.963593  33.331451  6.410217        1

[5 rows x 9 columns]

Shape:
(1554701, 9)

dtypes:
c_ax       float64
c_ay       float64
c_az       float64
c_ecg      float64
c_emg      float64
c_eda      float64
c_temp     float64
c_resp     float64
w_label      int64
dtype: object

Value Counts:
1    826000
2    471101
3    257600
Name: w_label, dtype: int64

          c_ecg     c_emg     c_eda     c_temp    c_resp  w_label
68859  0.030533  0.016342  0.988770  33.376709  6.391907        1
68860  0.044449 -0.008194  0.972366  33.3

In [ ]:
print('Baseline:')
s16_base = s16_final.loc[s16_final['w_label'].isin([1])]
print(s16_base.shape)
s16_basex = s16_base.drop('w_label' ,axis = 1)
print(s16_basex.shape)
print()
print('Stress:')
s16_stress = s16_final.loc[s16_final['w_label'].isin([2])]
print(s16_stress.shape)
s16_stressx = s16_stress.drop('w_label',axis = 1)
print(s16_stressx.shape)
print()
print('Amusement:')
s16_amuse = s16_final.loc[s16_final['w_label'].isin([3])]
print(s16_amuse.shape)
s16_amusex = s16_amuse.drop('w_label',axis =1)
print(s16_amusex.shape)

Baseline:
(826000, 6)
(826000, 5)

Stress:
(471101, 6)
(471101, 5)

Amusement:
(257600, 6)
(257600, 5)


In [ ]:
print('Creating windows:')
print()
print('Baseline:')
s16_basexw, count = window(s16_basex , 3500 ,1400)
print(count)
print(len(s16_basexw))

s16_basexwa , s16_basey  = data_ready(s16_basexw , count , 1)
print(s16_basexwa.shape)
print(s16_basey.shape)

print()
print('Stress:')
s16_stressxw, count = window(s16_stressx , 3500 ,1400)
print(count)
print(len(s16_stressxw))
s16_stressxwa , s16_stressy  = data_ready(s16_stressxw , count , 2)
print(s16_stressxwa.shape)
print(s16_stressy.shape)

print()
print('Amusement:')
s16_amusexw, count = window(s16_amusex , 3500 ,1400)
print(count)
print(len(s16_amusexw))
s16_amusexwa , s16_amusey  = data_ready(s16_amusexw , count , 3)
print(s16_amusexwa.shape)
print(s16_amusey.shape)

Creating windows:

Baseline:
588
588
(588, 5, 3500)
(588,)

Stress:
335
335
(335, 5, 3500)
(335,)

Amusement:
182
182
(182, 5, 3500)
(182,)


In [ ]:
x_s16 = np.concatenate( (s16_basexwa, s16_stressxwa, s16_amusexwa) , axis = 0)
print(len(x_s16))
print(x_s16.shape)
s16_x =  x_s16.reshape(x_s16.shape[0], 5, 3500, 1)
print(s16_x.shape)

1105
(1105, 5, 3500)
(1105, 5, 3500, 1)


In [ ]:
y_s16 = np.concatenate( (s16_basey, s16_stressy, s16_amusey) , axis = 0)
s16_y = pd.DataFrame(y_s16)

print(s16_y.shape)
print()
print(s16_y.columns)
print(s16_y.value_counts())
print()
print(s16_y.head())

(1105, 1)

RangeIndex(start=0, stop=1, step=1)
1    588
2    335
3    182
dtype: int64

   0
0  1
1  1
2  1
3  1
4  1


# Combining all the subjects

## Training set

In [ ]:
#1

train_x = np.concatenate( (s2_x, s3_x, s4_x) , axis = 0)
train_x.shape
train_y = np.concatenate( (s2_y, s3_y, s4_y) , axis = 0)
print(train_y.shape)
print(pd.DataFrame(train_y).value_counts())


'\ntrain_x = np.concatenate( (s2_x, s3_x, s4_x) , axis = 0)\ntrain_x.shape\ntrain_y = np.concatenate( (s2_y, s3_y, s4_y) , axis = 0)\nprint(train_y.shape)\nprint(pd.DataFrame(train_y).value_counts())\n'

In [ ]:

from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)
print(x_train1.shape)
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_test1[0:10])


'\nfrom sklearn.model_selection import train_test_split\nx_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)\nprint(x_train1.shape)\nprint(x_test1.shape)\nprint(y_train1.shape)\nprint(y_test1.shape)\nprint(y_test1[0:10])\n'

In [ ]:

xtr1 = np.concatenate((x_train1 , x_test1),axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr1.npy", xtr1)
print(xtr1.shape)
ytr1 = np.concatenate((y_train1, y_test1) , axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr1.npy", ytr1)
print(ytr1.shape)


'\nxtr1 = np.concatenate((x_train1 , x_test1),axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/xtr1.npy", xtr1)\nprint(xtr1.shape)\nytr1 = np.concatenate((y_train1, y_test1) , axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/ytr1.npy", ytr1)\nprint(ytr1.shape)\n'

In [ ]:
#2

train_x = np.concatenate( (s5_x, s6_x, s7_x) , axis = 0)
print(train_x.shape)
train_y = np.concatenate( (s5_y, s6_y, s7_y) , axis = 0)
print(train_y.shape)
print(pd.DataFrame(train_y).value_counts())


'\ntrain_x = np.concatenate( (s5_x, s6_x, s7_x) , axis = 0)\nprint(train_x.shape)\ntrain_y = np.concatenate( (s5_y, s6_y, s7_y) , axis = 0)\nprint(train_y.shape)\nprint(pd.DataFrame(train_y).value_counts())\n'

In [ ]:

from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)
print(x_train1.shape)
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_test1[0:10])


'\nfrom sklearn.model_selection import train_test_split\nx_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)\nprint(x_train1.shape)\nprint(x_test1.shape)\nprint(y_train1.shape)\nprint(y_test1.shape)\nprint(y_test1[0:10])\n'

In [ ]:

xtr2 = np.concatenate((x_train1 , x_test1),axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr2.npy", xtr2)
print(xtr2.shape)
ytr2 = np.concatenate((y_train1, y_test1) , axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr2.npy", ytr2)
print(ytr2.shape)


'\nxtr2 = np.concatenate((x_train1 , x_test1),axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/xtr2.npy", xtr2)\nprint(xtr2.shape)\nytr2 = np.concatenate((y_train1, y_test1) , axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/ytr2.npy", ytr2)\nprint(ytr2.shape)\n'

In [ ]:
#3

train_x = np.concatenate( (s8_x, s9_x, s10_x) , axis = 0)
print(train_x.shape)
train_y = np.concatenate( (s8_y, s9_y, s10_y) , axis = 0)
print(train_y.shape)
print(pd.DataFrame(train_y).value_counts())


'\ntrain_x = np.concatenate( (s8_x, s9_x, s10_x) , axis = 0)\nprint(train_x.shape)\ntrain_y = np.concatenate( (s8_y, s9_y, s10_y) , axis = 0)\nprint(train_y.shape)\nprint(pd.DataFrame(train_y).value_counts())\n'

In [ ]:

from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)
print(x_train1.shape)
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_test1[0:10])


'\nfrom sklearn.model_selection import train_test_split\nx_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)\nprint(x_train1.shape)\nprint(x_test1.shape)\nprint(y_train1.shape)\nprint(y_test1.shape)\nprint(y_test1[0:10])\n'

In [ ]:

xtr3 = np.concatenate((x_train1 , x_test1),axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr3.npy", xtr3)
print(xtr3.shape)
ytr3 = np.concatenate((y_train1, y_test1) , axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr3.npy", ytr3)
print(ytr3.shape)


'\nxtr3 = np.concatenate((x_train1 , x_test1),axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/xtr3.npy", xtr3)\nprint(xtr3.shape)\nytr3 = np.concatenate((y_train1, y_test1) , axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/ytr3.npy", ytr3)\nprint(ytr3.shape)\n'

In [ ]:
#4

train_x = np.concatenate( (s11_x, s13_x, s14_x) , axis = 0)
print(train_x.shape)
train_y = np.concatenate( (s11_y, s13_y, s14_y) , axis = 0)
print(train_y.shape)
print(pd.DataFrame(train_y).value_counts())


'\ntrain_x = np.concatenate( (s11_x, s13_x, s14_x) , axis = 0)\nprint(train_x.shape)\ntrain_y = np.concatenate( (s11_y, s13_y, s14_y) , axis = 0)\nprint(train_y.shape)\nprint(pd.DataFrame(train_y).value_counts())\n'

In [ ]:

from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)
print(x_train1.shape)
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_test1[0:10])


'\nfrom sklearn.model_selection import train_test_split\nx_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)\nprint(x_train1.shape)\nprint(x_test1.shape)\nprint(y_train1.shape)\nprint(y_test1.shape)\nprint(y_test1[0:10])\n'

In [ ]:

xtr4 = np.concatenate((x_train1 , x_test1),axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr4.npy", xtr4)
print(xtr4.shape)
ytr4 = np.concatenate((y_train1, y_test1) , axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr4.npy", ytr4)
print(ytr4.shape)


'\nxtr4 = np.concatenate((x_train1 , x_test1),axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/xtr4.npy", xtr4)\nprint(xtr4.shape)\nytr4 = np.concatenate((y_train1, y_test1) , axis = 0)\nnp.save("/content/drive/MyDrive/Ghanapriya Ma\'am/WESAD/ytr4.npy", ytr4)\nprint(ytr4.shape)\n'

In [ ]:
#5
train_x = np.concatenate( (s15_x, s16_x) , axis = 0)
print(train_x.shape)
train_y = np.concatenate( (s15_y, s16_y) , axis = 0)
print(train_y.shape)
print(pd.DataFrame(train_y).value_counts())

(2216, 5, 3500, 1)
(2216, 1)
1    1174
2     676
3     366
dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split( train_x , train_y, test_size=0.05, random_state=10)
print(x_train1.shape)
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_test1[0:10])

(2105, 5, 3500, 1)
(111, 5, 3500, 1)
(2105, 1)
(111, 1)
[[3]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]]


In [ ]:
xtr5 = np.concatenate((x_train1 , x_test1),axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr5.npy", xtr5)
print(xtr5.shape)
ytr5 = np.concatenate((y_train1, y_test1) , axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr5.npy", ytr5)
print(ytr5.shape)

(2216, 5, 3500, 1)
(2216, 1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_yle = le.fit_transform(train_y)
print(list(le.classes_))
print()
print(list(le.transform([1,2,3])))

In [ ]:
train_yfinal = np_utils.to_categorical(train_yle)
train_yfinal.shape

# Preparing the training set

In [ ]:
xtr1 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr1.npy")
print(xtr1.shape)
xtr2 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr2.npy")
print(xtr2.shape)
xtr3 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr3.npy")
print(xtr3.shape)
xtr4 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr4.npy")
print(xtr4.shape)
xtr5 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/xtr5.npy")
print(xtr5.shape)

In [ ]:
ytr1 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr1.npy")
print(ytr1.shape)
ytr2 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr2.npy")
print(ytr2.shape)
ytr3 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr3.npy")
print(ytr3.shape)
ytr4 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr4.npy")
print(ytr4.shape)
ytr5 = np.load("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/ytr5.npy")
print(ytr5.shape)

In [ ]:
x_train = np.concatenate((xtr1 , xtr2 , xtr3 , xtr4 , xtr5), axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/x_train.npy", x_train)
print(x_train.shape)
y_train = np.concatenate((ytr1 , ytr2 , ytr3 , ytr4 , ytr5 ), axis = 0)
np.save("/content/drive/MyDrive/Ghanapriya Ma'am/WESAD/y_train.npy",  y_train)
print(y_train.shape)